<a href="https://colab.research.google.com/github/olgasem10/Unsafe/blob/main/Gpt_ForSequenceClassification_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.9MB 10.9MB/s 
     |████████████████████████████████| 3.2MB 41.2MB/s 
     |████████████████████████████████| 890kB 42.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=849cd1db8411730425b8b3d111669dfc6566e9491c10ccfd8871a748d4988c3a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [31]:
from transformers import GPT2ForSequenceClassification, GPT2TokenizerFast, Trainer, TrainingArguments

import torch
from torch.utils.data import Dataset, DataLoader

import pandas as pd

In [3]:
model = GPT2ForSequenceClassification.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2', num_labels=2)
tokenizer = GPT2TokenizerFast.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Some weights of the model checkpoint at sberbank-ai/rugpt3small_based_on_gpt2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/rugpt3small_based_on_gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
train = pd.read_csv('/content/drive/MyDrive/internship/data/train_randst0.csv')
test = pd.read_csv('/content/drive/MyDrive/internship/data/val_randst0.csv')
train = train[["text", "unsafe"]]
test = test[["text", "unsafe"]]

In [8]:
train = train.loc[(train['unsafe'] >= 0.8) | (train['unsafe'] <= 0.2)]

In [9]:
def binary(prob):
    if prob < 0.5:
        return 0.
    else:
        return 1.0

In [10]:
train['unsafe'] = train['unsafe'].apply(binary)
test['unsafe'] = test['unsafe'].apply(binary)

In [11]:
train

,text,unsafe
0,я думал что левиафаны - это те медленные страх...,1.0
1,"А был бы этот полицейский в Петербурге, так пе...",1.0
2,"Напоминаю, что пора искать актис невзрослого п...",1.0
3,курю лет пятнадцать никаких проблем кроме како...,1.0
4,"окей, я тогда проведу парад гетеросексуалов, п...",1.0
...,...,...
138825,"Перед клиентом отвечает банк, а не сотрудник.",0.0
138826,"Так воооот откуда я их знаю, какое старое виде...",0.0
138827,"Да потом просто ""такие вот люди"" начинают жало...",0.0
138829,"А теперь давай фоточки не миллионеров, а обычн...",0.0


In [12]:
train = train.sample(frac=1)

In [13]:
test

,text,unsafe
0,"уровень ссачнее, чем ад в доом 3",1.0
1,У нас несколько спортсменов и спортсменок в сп...,1.0
2,Я уж думал нахлебались праздников Запада ...а ...,1.0
3,"Лол, большая часть около NUMBER% и так на нога...",1.0
4,"Не, ну тут действительно в какой то степени ма...",1.0
...,...,...
24496,Почему все так вокруг Киану пляшут?,0.0
24497,"Проблема то не в том, что ты деньги проиграл. ...",0.0
24498,"Не не не, не говори. Пусть остается в блаженно...",0.0
24499,"На иврите будет אין חניה , а не то что там на...",0.0


In [14]:
x_train = train['text'].tolist()
y_train = train['unsafe'].tolist()
x_test = test['text']
y_test = test['unsafe']

In [15]:
len(x_train)

120120

In [16]:
x_train[1000], y_train[1000]

('у кого-нибудь есть тексты детских песен про мальчишек? Или хотя бы названия песен?',
 0.0)

In [18]:
max_len = 45

In [20]:
tokenizer.pad_token = '<pad>'

In [19]:
class UnsafeData(Dataset):

    def __init__(self, texts, targets, max_len):
        
        super().__init__()
        
        self.texts = texts
        self.targets = targets        
        self.max_len = max_len        

    def __len__(self):
        
        return len(self.texts)

    
    def __getitem__(self, index):
      
      x = self.texts[index]

      enc_dict = tokenizer(x, truncation=True, max_length=max_len, padding='max_length')
      
      item = {key: torch.tensor(val).long() for key, val in enc_dict.items()}
      item['labels'] = torch.tensor(self.targets[index]).long()

      return item 

In [21]:
train_dataset = UnsafeData(texts = x_train, targets = y_train, max_len=max_len)
test_dataset = UnsafeData(texts = x_test, targets = y_test, max_len=max_len)

In [22]:
len(train_dataset), len(test_dataset)

(120120, 24501)

In [23]:
train_dataset[23000]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([ 4182,   647,  4993,  3240, 38055,   329,  1304,    35,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]),
 'labels': tensor(0)}

In [29]:
device = torch.device('cuda')

In [30]:
model.to(device)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
      

In [32]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='/content/drive/My Drive/internship/gpt_3',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    #per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    #evaluate_during_training=True,
    logging_dir='/content/drive/My Drive/internship/gpt_3/logs',
    save_steps = 2000
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
len(train_dataset)

120120

In [33]:
model.config.pad_token_id = 0

In [ ]:
trainer.train()

Step,Training Loss
500,0.276600
1000,0.325100
1500,0.342600
2000,0.346500
2500,0.352000
3000,0.336000
3500,0.340200
4000,0.351800
4500,0.343300
5000,0.336900


TrainOutput(global_step=30030, training_loss=0.28903322095994827, metrics={'train_runtime': 3855.9917, 'train_samples_per_second': 7.788, 'total_flos': 8123223357849600, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'epoch': 2.0,
 'eval_accuracy': 0.7753969225745888,
 'eval_f1': 0.5946821831037784,
 'eval_loss': 0.9682791829109192,
 'eval_precision': 0.6512340700112922,
 'eval_recall': 0.5471672539983735,
 'eval_runtime': 81.0651,
 'eval_samples_per_second': 302.239}

In [ ]:
trainer.evaluate() # 3 18

{'eval_accuracy': 0.7765397330721195,
 'eval_f1': 0.5811978887783983,
 'eval_loss': 0.8764160871505737,
 'eval_precision': 0.6670763827919227,
 'eval_recall': 0.5149091894822445,
 'eval_runtime': 80.8252,
 'eval_samples_per_second': 303.136}

In [ ]:
trainer.evaluate() # 3 20

{'eval_accuracy': 0.7736827068282928,
 'eval_f1': 0.602935911206588,
 'eval_loss': 0.9891657829284668,
 'eval_precision': 0.639137695460756,
 'eval_recall': 0.5706153429113581,
 'eval_runtime': 80.134,
 'eval_samples_per_second': 305.75}

In [ ]:
trainer.evaluate() # 3 22

{'eval_accuracy': 0.7757234398595976,
 'eval_f1': 0.5874934314240672,
 'eval_loss': 0.9314336180686951,
 'eval_precision': 0.6584216725559482,
 'eval_recall': 0.530360531309298,
 'eval_runtime': 80.2002,
 'eval_samples_per_second': 305.498}

In [ ]:
trainer.evaluate() # 3 24

{'eval_accuracy': 0.7756826251989715,
 'eval_f1': 0.6053425247738045,
 'eval_loss': 0.8898417353630066,
 'eval_precision': 0.6437080024434942,
 'eval_recall': 0.5712930333423692,
 'eval_runtime': 79.392,
 'eval_samples_per_second': 308.608}

In [ ]:
trainer.evaluate() # 3 26

{'eval_accuracy': 0.7751928492714583,
 'eval_f1': 0.6067399685848922,
 'eval_loss': 1.0090335607528687,
 'eval_precision': 0.6410681955340978,
 'eval_recall': 0.5759013282732448,
 'eval_runtime': 78.9148,
 'eval_samples_per_second': 310.474}

In [ ]:
trainer.evaluate()

{'epoch': 2.0,
 'eval_accuracy': 0.787273988816783,
 'eval_f1': 0.6235735952621696,
 'eval_loss': 0.5967862010002136,
 'eval_precision': 0.6674397031539888,
 'eval_recall': 0.5851179181349959,
 'eval_runtime': 80.497,
 'eval_samples_per_second': 304.371}